In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("diabetes.csv")
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Outcome'] = le.fit_transform(data['Outcome'])
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)

In [3]:
class_labels = np.unique(Y) 
gini = 0 
for cls in class_labels:
    p_cls = len(Y[Y == cls]) / len(Y) 
    gini += p_cls ** 2 
print(1 - gini) 

0.45437282986111116


In [6]:
def gini_index(y):
    class_labels = np.unique(y) # take unique values from target column
    gini = 0 # initialize gini value with 0
    for cls in class_labels:
        p_cls = len(y[y == cls]) / len(y) # draw values from class labels and get the probablity
        gini += p_cls ** 2 # using gini index formula and update after every iteration
    return 1 - gini 

In [14]:
dataset = data.to_numpy()
X, Y = dataset[:, :-1], dataset[:, -1]
n_rows, n_columns = np.shape(X)
best_split = {}
max_info_gain = -1

# loop over all the features
for f_idxs in range(n_columns):
    feature_values = dataset[:, f_idxs]
    possible_thresholds = np.unique(feature_values)
    # loop over all the feature values present in the data
    for thrs in possible_thresholds:
        # get current split
        left_idxs = np.array([row for row in dataset if row[f_idxs] <= thrs])
        right_idxs = np.array([row for row in dataset if row[f_idxs] > thrs])
        # check if childs are not null
        if len(left_idxs) > 0 and len(right_idxs) > 0:
            y, left_y, right_y = dataset[:, -1], left_idxs[:, -1], right_idxs[:, -1]
            # compute information gain
            weight_l = len(left_y) / len(y)
            weight_r = len(right_y) / len(y)
            curr_info_gain = gini_index(y) - (weight_l * gini_index(left_y) + weight_r * gini_index(right_y))
            # update the best split if needed
            if curr_info_gain > max_info_gain:
                best_split["feature_index"] = f_idxs
                best_split["threshold"] = thrs
                best_split["dataset_left"] = left_idxs
                best_split["dataset_right"] = right_idxs
                best_split["info_gain"] = curr_info_gain
                max_info_gain = curr_info_gain
print(best_split)

{'feature_index': 0, 'threshold': 6.0, 'dataset_left': array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  1.   ,  89.   ,  66.   , ...,   0.167,  21.   ,   0.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]]), 'dataset_right': array([[8.00e+00, 1.83e+02, 6.40e+01, ..., 6.72e-01, 3.20e+01, 1.00e+00],
       [1.00e+01, 1.15e+02, 0.00e+00, ..., 1.34e-01, 2.90e+01, 0.00e+00],
       [8.00e+00, 1.25e+02, 9.60e+01, ..., 2.32e-01, 5.40e+01, 1.00e+00],
       ...,
       [9.00e+00, 1.70e+02, 7.40e+01, ..., 4.03e-01, 4.30e+01, 1.00e+00],
       [9.00e+00, 8.90e+01, 6.20e+01, ..., 1.42e-01, 3.30e+01, 0.00e+00],
       [1.00e+01, 1.01e+02, 7.60e+01, ..., 1.71e-01, 6.30e+01, 0.00e+00]]), 'info_gain': 0.025641862239203506}


In [7]:
def get_best_split(dataset, num_features):

    best_split = {} #create a dictionary to load best split values
    max_info_gain = -1 # initialize info gain value with -1

    for feature_index in range(num_features):
        feature_values = dataset[:, feature_index]
        possible_thresholds = np.unique(feature_values) # draw the unique values from all the features and load it to a variable

        for threshold in possible_thresholds:
            dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold]) # spliting the left tree and right tree for every to the threshold
            dataset_right = np.array([row for row in dataset if row[feature_index] > threshold]) # spliting the left tree and right tree for every to the threshold
            if len(dataset_left) > 0 and len(dataset_right) > 0: # check if child nodes are not null
                y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1] # load all the dependent values of parent, left and right child into three different variable
                weight_l = len(left_y) / len(y) # calculating weighted average left child
                weight_r = len(right_y) / len(y) # calculating weighted average right child
                curr_info_gain = gini_index(y) - (weight_l * gini_index(left_y) + weight_r * gini_index(right_y)) # obtaining information gain value
                # update the best split if current info gain is higher than previous value 
                if curr_info_gain > max_info_gain: 
                    best_split["feature_index"] = feature_index
                    best_split["threshold"] = threshold
                    best_split["dataset_left"] = dataset_left
                    best_split["dataset_right"] = dataset_right
                    best_split["info_gain"] = curr_info_gain
                    max_info_gain = curr_info_gain

    return best_split

In [11]:
curr_depth=0
min_samples_split=3
max_depth=3

In [9]:
dataset = data.to_numpy()
X, Y = dataset[:, :-1], dataset[:, -1]
n_rows, n_columns = np.shape(X)

if n_rows >= min_samples_split and curr_depth <= max_depth: # checking for stoping criteria
    best_split = get_best_split(dataset, n_columns) 
    if best_split["info_gain"] > 0: 
        left_subtree = (best_split["dataset_left"], curr_depth + 1) # recur left tree until stopping criteria is met 
        right_subtree = (best_split["dataset_right"], curr_depth + 1) # recur right tree until stopping criteria is met 
        print(best_split["feature_index"], best_split["threshold"],left_subtree, right_subtree, best_split["info_gain"])

Y = list(Y)
leaf_value = max(Y, key=Y.count)

<built-in method count of list object at 0x7f1e622807c0>


In [33]:
Node={}
value=None
def build_tree(dataset, curr_depth=0):
        ''' recursive function to build the tree '''

        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        # split until stopping conditions are met
        if num_samples >= min_samples_split and curr_depth <=max_depth:
            # find the best split
            best_split = get_best_split(dataset,num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # recur left nodes
                left_subtree = build_tree(best_split["dataset_left"], curr_depth + 1)
                # recur right nodes
                right_subtree = build_tree(best_split["dataset_right"], curr_depth + 1)
                # return decision node
                return best_split["feature_index"], best_split["threshold"],left_subtree, right_subtree, best_split["info_gain"]

        # compute leaf node
        Y = list(Y)
        leaf_value = max(Y, key=Y.count)
        value=leaf_value      
        return value


In [12]:
dataset = data.to_numpy()

In [34]:
root = build_tree(dataset,curr_depth=0)

In [40]:
root

(1,
 127.0,
 (7,
  28.0,
  (5,
   45.3,
   (5, 30.9, 0.0, 0.0, 0.009898623023798292),
   (1, 115.0, 1.0, 0.0, 0.375),
   0.013255334468846858),
  (5,
   26.2,
   (5, 0.0, 1.0, 0.0, 0.09280190362879237),
   (1, 99.0, 0.0, 0.0, 0.043906943402416077),
   0.03795997591707889),
  0.030059990043791396),
 (5,
  29.9,
  (1,
   145.0,
   (4, 130.0, 0.0, 0.0, 0.019886122206169754),
   (7, 24.0, 0.0, 1.0, 0.06825543120474004),
   0.06726958603183181),
  (1,
   157.0,
   (7, 30.0, 0.0, 1.0, 0.0340159953468081),
   (4, 579.0, 1.0, 0.0, 0.01938503147058593),
   0.03360638521319054),
  0.06566973418273542),
 0.08250014459160077)